### Prediction the model 

In [1]:
import tensorflow as tf
from  tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [32]:
## load model,scaler, onhotEncoder,Label Encoder

model = load_model("model.h5")

with open("scaler.pkl","rb") as file:
    scaler = pickle.load(file)
with open("label_encoder_gender.pkl","rb") as file:
    label_encoder_gender = pickle.load(file)

with open("oneHotencoder_geo.pkl","rb") as file:
    label_encoder_geo = pickle.load(file)        


In [41]:
input_data = {
    "CreditScore":600 ,
    "Geography":'France',
    "Gender":'Male' ,
    "Age":40,
    "Tenure":3,
    "Balance":60000,
    "NumOfProducts":2,
    "HasCrCard":1,
    "IsActiveMember":1,
    "EstimatedSalary":50000,
}

In [42]:
## onhot encoder
geo_encoder = label_encoder_geo.transform([[input_data["Geography"]]]).toarray()


c:\Users\vipin\OneDrive\Desktop\DL\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [43]:
label_encoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [44]:
geo_encoder_df = pd.DataFrame(geo_encoder,columns=label_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoder_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [52]:
## input_data into dict to dataframe
input_df = pd.DataFrame([input_data])
input_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [54]:
input_df["Gender"] = label_encoder_gender.transform(input_df[["Gender"]])
input_df.head()

c:\Users\vipin\OneDrive\Desktop\DL\venv\Lib\site-packages\sklearn\preprocessing\_label.py:139: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [55]:
## concat 

input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoder_df],axis=1)
input_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [56]:
## standarerize

input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53778968,  0.91205588,  0.10469697, -0.69111911, -0.26118783,
         0.80881484,  0.65065941,  0.97093502, -0.87571845,  1.00308167,
        -0.57912674, -0.57794242]])

In [61]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [63]:
## model pridiction

prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


array([[0.0296176]], dtype=float32)

In [64]:
prediction_prob = prediction[0][0]
prediction_prob

np.float32(0.029617598)

In [65]:
if prediction_prob > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')


The customer is not likely to churn.
